In [1]:
# import basic modules
import sys
import os
import numpy as np
import pandas as pd
import PIL

from utils import default_paths, nsd_utils, prf_utils, segmentation_utils
from model_fitting import initialize_fitting

# import coco api tools
coco_api_path = default_paths.coco_api_path
sys.path.append(os.path.join(coco_api_path,'cocoapi','PythonAPI'))
from pycocotools.coco import COCO


In [44]:
size_fn = '/user_data/mmhender/nsd/labels/S1_within_prf_grid5/S1_realworldsize_prf1455.csv'
size_df = pd.read_csv(size_fn, index_col=0)
size_df

,has_small,has_medium,has_large
0,True,True,False
1,True,True,True
2,False,True,False
3,False,True,True
4,True,False,False
...,...,...,...
9995,True,True,False
9996,True,True,True
9997,True,True,False
9998,False,True,True


In [48]:
labels = np.array(size_df)[:,[0,2]].astype(np.float32)
colnames = [size_df.keys()[0], size_df.keys()[2]]
    

In [50]:
colnames

['has_small', 'has_large']

In [49]:
labels

array([[1., 0.],
       [1., 1.],
       [0., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 1.]], dtype=float32)

In [39]:
names_to_sizes = np.load(fn2save, allow_pickle=True).item()
names_to_sizes

{'person': 1,
 'bicycle': 1,
 'car': 2,
 'motorcycle': 1,
 'airplane': 2,
 'bus': 2,
 'train': 2,
 'truck': 2,
 'boat': 2,
 'traffic light': 1,
 'fire hydrant': 1,
 'stop sign': 1,
 'parking meter': 1,
 'bench': 2,
 'bird': 0,
 'cat': 0,
 'dog': 0,
 'horse': 2,
 'sheep': 1,
 'cow': 2,
 'elephant': 2,
 'bear': 2,
 'zebra': 2,
 'giraffe': 2,
 'backpack': 0,
 'umbrella': 0,
 'handbag': 0,
 'tie': 0,
 'suitcase': 1,
 'frisbee': 0,
 'skis': 1,
 'snowboard': 1,
 'sports ball': 0,
 'kite': 1,
 'baseball bat': 0,
 'baseball glove': 0,
 'skateboard': 0,
 'surfboard': 1,
 'tennis racket': 0,
 'bottle': 0,
 'wine glass': 0,
 'cup': 0,
 'fork': 0,
 'knife': 0,
 'spoon': 0,
 'bowl': 0,
 'banana': 0,
 'apple': 0,
 'sandwich': 0,
 'orange': 0,
 'broccoli': 0,
 'carrot': 0,
 'hot dog': 0,
 'pizza': 0,
 'donut': 0,
 'cake': 0,
 'chair': 2,
 'couch': 2,
 'potted plant': 0,
 'bed': 2,
 'dining table': 2,
 'toilet': 1,
 'tv': 1,
 'laptop': 0,
 'mouse': 0,
 'remote': 0,
 'keyboard': 0,
 'cell phone': 0,
 '

In [40]:
# make [3 x ncateg] one-hot array of size labels for each categ
categ_size_labels = np.array([[names_to_sizes[kk]==ii for kk in names_to_sizes.keys()] \
                          for ii in range(3)])

In [42]:
# print size lists as a sanity check
for ii in range(3):
    print('things categories with size %d:'%ii)
    print(np.array(cat_names)[categ_size_labels[ii,:]])


things categories with size 0:
['bird' 'cat' 'dog' 'backpack' 'umbrella' 'handbag' 'tie' 'frisbee'
 'sports ball' 'baseball bat' 'baseball glove' 'skateboard'
 'tennis racket' 'bottle' 'wine glass' 'cup' 'fork' 'knife' 'spoon' 'bowl'
 'banana' 'apple' 'sandwich' 'orange' 'broccoli' 'carrot' 'hot dog'
 'pizza' 'donut' 'cake' 'potted plant' 'laptop' 'mouse' 'remote'
 'keyboard' 'cell phone' 'toaster' 'book' 'clock' 'vase' 'scissors'
 'teddy bear' 'hair drier' 'toothbrush']
things categories with size 1:
['person' 'bicycle' 'motorcycle' 'traffic light' 'fire hydrant'
 'stop sign' 'parking meter' 'sheep' 'suitcase' 'skis' 'snowboard' 'kite'
 'surfboard' 'toilet' 'tv' 'microwave' 'oven' 'sink']
things categories with size 2:
['car' 'airplane' 'bus' 'train' 'truck' 'boat' 'bench' 'horse' 'cow'
 'elephant' 'bear' 'zebra' 'giraffe' 'chair' 'couch' 'bed' 'dining table'
 'refrigerator']
